In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='d1eb76fe-31c6-4943-893d-ea826b35e138', project_access_token='p-9a87f0903e74226240ae8cbaf3beeade92a64aba')
pc = project.project_context


# Segmenting and Clustering Neighborhoods in Toronto part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

#### Importing libraries and transforming pandas into df

In [2]:
# lib to handle request
import requests
# lib to data analysis
import pandas as pd
# lib to vector data as arrays
import numpy as np
# lib for random # generator
import random 
# lib to display images
from IPython.display import Image
from IPython.core.display import HTML
# lib for plotting
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# module of address into lat and lon values
!conda install -c conda-forge geopy --yes 

# transform json to pd lib
from pandas.io.json import json_normalize

print('Folium ok')
print('Lib ok')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium ok
Lib ok


In [3]:
!conda install -c conda-forge geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


NameError: name 'postal_code' is not defined

In [4]:
import pandas as pd 
import numpy as np

link = "http://cocl.us/Geospatial_data"
df = pd.read_csv(link)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df.shape

(103, 3)

In [6]:
# change the columns name to merge 

df.columns = ['Postcode', 'Latitude', 'Longitude']

cols = df.columns.tolist()
cols

['Postcode', 'Latitude', 'Longitude']

In [7]:
link = "https://raw.githubusercontent.com/edward0rtiz/coursera_capstone/master/df_can.csv"
df2 = pd.read_csv(link,index_col=0)
df2.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge ,Malvern"
M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [8]:
# merging process

df_mer = pd.merge(df2, df, on='Postcode')
df_mer.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
# save
project.save_data(data=df_mer.to_csv(index=False),file_name='df_can2.csv',overwrite=True)

{'file_name': 'df_can2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datascience-donotdelete-pr-1gtwsm1beg0vbi',
 'asset_id': '77343a4a-e5cb-48fc-a1e7-257534348a7f'}